# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
html = requests.get(url).content

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(html,"lxml")

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [5]:
domain = 'http://wikipedia.org'

In [6]:
list_link=[]

link_rela = [l.get('href') for l in (soup.find_all('a'))]
rela = ['http:'+l if str(l).startswith('//')==True else l for l in link_rela ]
rela = [domain+l if str(l).startswith('/')==True else l for l in link_rela]
[list_link.append(r) for r in rela if str(r).startswith('http')]
list_link

['http://wikipedia.org/wiki/Information_science',
 'http://wikipedia.org/wiki/Machine_learning',
 'http://wikipedia.org/wiki/Data_mining',
 'http://wikipedia.org/wiki/File:Kernel_Machine.svg',
 'http://wikipedia.org/wiki/Statistical_classification',
 'http://wikipedia.org/wiki/Cluster_analysis',
 'http://wikipedia.org/wiki/Regression_analysis',
 'http://wikipedia.org/wiki/Anomaly_detection',
 'http://wikipedia.org/wiki/Automated_machine_learning',
 'http://wikipedia.org/wiki/Association_rule_learning',
 'http://wikipedia.org/wiki/Reinforcement_learning',
 'http://wikipedia.org/wiki/Structured_prediction',
 'http://wikipedia.org/wiki/Feature_engineering',
 'http://wikipedia.org/wiki/Feature_learning',
 'http://wikipedia.org/wiki/Online_machine_learning',
 'http://wikipedia.org/wiki/Semi-supervised_learning',
 'http://wikipedia.org/wiki/Unsupervised_learning',
 'http://wikipedia.org/wiki/Learning_to_rank',
 'http://wikipedia.org/wiki/Grammar_induction',
 'http://wikipedia.org/wiki/Superv

In [7]:
abso=[l for l in link_rela if str(l).startswith('http')==True]
[list_link.append(a) for a in abso]

temp =[]
for x in list_link:   
    if '%' in x:
        list_link.remove(x)
    else : 
        temp.append(x)
        
# function to get unique values 
def unique(list1):     
    # insert the list to the set 
    list_set = set(list1) 
    # convert the set to the list 
    unique_list = (list(list_set)) 
    return unique_list      
        
unique_links = unique(temp)
#unique_links.sort()
#unique_links


## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [8]:
import os

In [9]:
try:
    os.mkdir('wikipedia')
except:
    print('il y a un problème')

In [10]:
os.chdir('wikipedia')

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [11]:
from slugify import slugify

In [12]:
def index_page(link):
    slug_soup = BeautifulSoup(requests.get(link).content)
    slug_content = slug_soup.text
    slug = (slugify(link)+'.html')
    
    slug_file = open(slug, "w") #wb permet d'avoir une page en html jolie
    try : 
        slug_file.write(slug_content)
    except OSError as e:
        pass
    slug_file.close()


#index_page(domain)

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [13]:
%%time
for link in unique_links:
    index_page(link)

CPU times: user 47.8 s, sys: 1.91 s, total: 49.7 s
Wall time: 2min 42s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [14]:
import multiprocessing

In [15]:
pool = multiprocessing.Pool()

In [16]:
%%time
result = pool.map(index_page, unique_links)
pool.terminate()
pool.join()


CPU times: user 32 ms, sys: 12.9 ms, total: 44.9 ms
Wall time: 41.6 s
